In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
import nltk


**Install the SDK**

In [2]:
%pip install -U -q "google-generativeai>=0.8.5"

Note: you may need to restart the kernel to use updated packages.


**Setting up API key**

To use the Google AI API in my Kaggle notebook, I'll need to obtain an API key from AI Studio. Once I have the key, I'll store it securely as a Kaggle secret named GOOGLE_API_KEY. This will enable me to access and utilize the API's powerful capabilities within your notebook.

In [3]:
import google.generativeai as genai
from IPython.display import HTML, Markdown, display

In [4]:
from kaggle_secrets import UserSecretsClient

GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
genai.configure(api_key=GOOGLE_API_KEY)

**Running Prompt**

In [5]:
from IPython.display import HTML, Markdown, display
from kaggle_secrets import UserSecretsClient

flash = genai.GenerativeModel('gemini-1.5-flash')
response = flash.generate_content("Tell me something interesting about quantum computing.")
print(response.text)

One fascinating aspect of quantum computing is its potential to **break widely used encryption methods**.  Current internet security relies heavily on the difficulty of factoring very large numbers – a problem that's computationally expensive for classical computers. However, Shor's algorithm, a quantum algorithm, can solve this problem exponentially faster, potentially rendering much of our online security obsolete.  This highlights the double-edged sword of quantum computing: its immense power carries significant implications for both security and vulnerability.



In [6]:
Markdown(response.text)

One fascinating aspect of quantum computing is its potential to **break widely used encryption methods**.  Current internet security relies heavily on the difficulty of factoring very large numbers – a problem that's computationally expensive for classical computers. However, Shor's algorithm, a quantum algorithm, can solve this problem exponentially faster, potentially rendering much of our online security obsolete.  This highlights the double-edged sword of quantum computing: its immense power carries significant implications for both security and vulnerability.


**Building a Restaurant Conversational Agent: Enhancing the Dining Experience**

We're developing a sophisticated conversational agent designed to revolutionize the dining experience. This AI-powered bot will provide real-time assistance to customers, from menu recommendations to order placement and payment processing.

**Data**

Here is a small set of documents I will be using to create an embedding database.

In [7]:
Document1 = """Our restaurant is called "The Golden Spoon." We are located at 123 Main Street in Varanasi, Uttar Pradesh. You can reach us by phone at 0542-XXXXXXX. Our website is www.thegoldenspoon.com. We offer limited street parking. The main entrance and dining area are wheelchair accessible. We have a charming outdoor seating area during pleasant weather. We specialize in authentic North Indian cuisine with a modern twist. Ask us about our daily specials!"""

Document2 = """Welcome to The Golden Spoon in Varanasi! We offer a diverse menu featuring the best of Indian, Chinese, Continental, South Indian, and Italian cuisines. Our opening hours are from 11:00 AM to 11:00 PM daily. For reservations or inquiries, please call us at 0542-XXXXXXX or visit our website at www.thegoldenspoon.com.

**Our Menu:**

**Indian Delights:**
Our Indian menu boasts popular dishes like creamy Butter Chicken (₹380), rich Paneer Tikka Masala (₹350), fragrant Chicken Biryani (₹420), flavorful Dal Makhani (₹300), and spicy Chana Masala (₹280). We also offer a variety of Indian breads including Naan (₹50), Roti (₹30), and Paratha (₹70). For seafood lovers, our Goan Fish Curry (₹450) is a must-try.

**Chinese Favorites:**
Indulge in our classic Chinese dishes such as Chicken Manchurian (dry/gravy - ₹320), Vegetable Fried Rice (₹250), Hakka Noodles (₹280), Schezwan Chicken (₹350), and Spring Rolls (₹180 for 2). We also offer a spicy Kung Pao Chicken (₹360) and a comforting Sweet and Sour Vegetables (₹290).

**Continental Selection:**
Our Continental menu features Grilled Chicken with Mushroom Sauce (₹400), Fish and Chips (₹380), Shepherd's Pie (₹360), Vegetable Au Gratin (₹320), and Chicken Stroganoff (₹420). These dishes offer a taste of global flavors prepared with our special touch.

**South Indian Specialties:**
Experience the authentic flavors of South India with our crispy Dosa (plain - ₹120, masala - ₹180, paneer - ₹220), fluffy Idli with Sambar (₹150 for 2), savory Vada (₹100 for 2), and tangy Pongal (₹160). Our South Indian Thali (₹300) offers a delightful assortment of regional specialties.

**Italian Indulgence:**
Savor our Italian classics including Margherita Pizza (₹350 for 10 inch), creamy Alfredo Pasta (with chicken - ₹400, with vegetables - ₹350), spicy Arrabbiata Pasta (₹320), cheesy Lasagna (₹450), and flavorful Mushroom Risotto (₹380). Our Garlic Bread (₹120 for 4 slices) is a perfect accompaniment.

**Beverages:**
We offer a range of beverages including Soft Drinks (₹60), Juices (₹80), Lassi (Sweet/Salted - ₹90, Mango - ₹120), Masala Chai (₹50), and Coffee (₹70).

**Desserts:**
Our dessert selection includes traditional Indian sweets like Gulab Jamun (₹80 for 2) and Rasgulla (₹70 for 2), as well as international favorites like Chocolate Lava Cake (₹200) and Ice Cream (₹100 per scoop).

**Vegetarian & Vegan Options:** We have a wide array of delicious vegetarian and vegan options across all cuisines, clearly marked on our in-house and online menus. Please ask your server for assistance.

**Allergies:** Please inform your server of any dietary restrictions or allergies, and our staff will be happy to assist you.

Enjoy your dining experience at The Golden Spoon! We appreciate your patronage.
"""

Document3 = """Yes, we do offer delivery within a 5-kilometer radius of our restaurant. The minimum order for delivery is ₹300. Our delivery charges are ₹50 for orders below ₹500 and free for orders above ₹500. The estimated delivery time is typically 30-45 minutes, depending on the distance and order volume. You can customize most dishes; please specify your requests when placing your order (e.g., "no onions," "extra spicy"). We accept online payments, credit/debit cards upon delivery, and cash on delivery. Gift cards can be redeemed for both dine-in and delivery orders. To inquire about the status of your delivery or to request a cancellation (within a reasonable timeframe before dispatch), please call us immediately at 0542-XXXXXXX."""

Document4 = """Yes, we highly recommend making a reservation, especially during peak hours and weekends. You can make a reservation through our website at www.thegoldenspoon.com/reservations or by calling us at 0542-XXXXXXX. The earliest reservation we typically take is for our opening time, and the latest is one hour before closing. You can reserve a table for up to 10 people online. For larger parties, please call us directly. While we try our best to accommodate requests for specific tables, it cannot be guaranteed. The standard table reservation time is 1.5 to 2 hours, depending on the party size. To modify or cancel your reservation, please do so at least 2 hours in advance via our website or by phone."""

Document5 = """Our goal is to provide a delicious and satisfying dining experience. We use fresh, flavorful, and quality ingredients in all our dishes. Our chefs ensure that each meal is well-cooked and presented beautifully. If, for any reason, you find your food to be undercooked, overcooked, bland, or not to your satisfaction, please inform your server immediately, and we will do our best to rectify the situation. We strive for friendly, attentive, and efficient service. If you experience slow or rude service, please speak to our manager. We aim to create a cozy and comfortable ambiance with tasteful decor and background music. We believe our prices offer reasonable value for the quality of food and service provided. If you have any complaints or feedback regarding your overall experience, please don't hesitate to share it with our staff or leave a review on our website or other platforms."""

Document6 = """We take dietary needs and allergies seriously. Please inform your server about any vegetarian, vegan, or gluten-free requirements or any specific allergies, such as nut or dairy allergies. Our staff is trained to handle special requests and can guide you through the menu options that are suitable for you. While we take precautions to avoid cross-contamination, please be aware that our kitchen handles various allergens. We aim for order accuracy and apologize if you receive a wrong order or a missing item. Please notify us immediately, and we will promptly correct your order. For delivery experiences, we strive for on-time delivery and ensure that food is properly packaged. If your delivery is late, or if you receive cold food or damaged packaging, please contact us, and we will address your concerns."""

Document7 = """Our kitchen maintains a stock of fresh produce, various meats, dairy products, grains, and a wide array of spices and beverages to ensure we can prepare all our menu items. We regularly monitor our stock levels to avoid running out of ingredients. We work with trusted suppliers to order and purchase high-quality ingredients, and we maintain detailed invoices for all purchases. Our storage facilities include refrigeration, freezers, and dry storage areas to maintain the freshness and quality of our ingredients. We adhere to strict shelf life guidelines and manage our inventory to minimize spoilage and waste through careful portion control. We have a system in place for tracking our inventory, including regular counts and audits."""

Document8 = """We continuously strive to streamline our processes for efficiency. Our order management system ensures quick processing and fulfillment of both dine-in and delivery orders. Our kitchen utilizes a kitchen display system (KDS) to optimize workflow. For table management, our seating arrangements are designed for optimal flow, and we use a reservation system to manage bookings and waitlists. We offer various payment processing options, including online payments, card payments through our point of sale (POS) system, and cash payments. Internal and staff communication is crucial for smooth operations. We are exploring opportunities for automation, such as automated online ordering and digital menus. Our overall workflow is designed with clear steps and coordination among all team members. We leverage technology and integrate various software platforms to enhance our efficiency."""

Document9 = """We often have special offers and promotions. Please check our website or ask your server about our current deals and discounts. We also have a happy hour from 5 PM to 7 PM daily, featuring discounted beverages and appetizers. Yes, we do host private events and offer catering services. Our catering options are flexible and can be customized to suit your needs. Please contact our events coordinator for more details. We encourage our customers to leave feedback and reviews on our website and platforms like Google Reviews. Our ratings are important to us, and we continuously strive to improve based on customer feedback. We are currently developing a loyalty program to reward our regular customers. Stay tuned for more information on how to earn points and redeem rewards."""

Document10 = """For specific allergen information about any dish, please ask your server, and they can provide you with detailed information from our allergen menu. We are working on providing nutritional values for our menu items on our website soon. We are committed to sustainability and use eco-friendly packaging for our takeout and delivery orders where possible. We prioritize sourcing our ingredients locally whenever feasible to support local farmers and ensure freshness. Our kitchen staff includes experienced chefs who oversee the preparation of all dishes. Our friendly servers and waiters are here to provide excellent service at your table. Our welcoming hostess will assist you with seating arrangements. The restaurant is managed by a dedicated team focused on ensuring a positive dining experience. Our skilled bartenders prepare a variety of beverages at our bar. Our kitchen is equipped with modern equipment, including efficient ovens, stoves, fryers, and a commercial dishwasher. We utilize a POS system for order taking and payment processing."""

documents = [Document1, Document2, Document3, Document4, Document5, Document6, Document7, Document8, Document9, Document10]

**Setup**

First, install ChromaDB and the Gemini API Python SDK.

In [8]:
%pip install -U -q "google-generativeai>=0.8.5" chromadb

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kfp 2.5.0 requires google-cloud-storage<3,>=2.2.1, but you have google-cloud-storage 1.44.0 which is incompatible.
kfp 2.5.0 requires kubernetes<27,>=8.0.0, but you have kubernetes 32.0.1 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


**ChromaDB**

Create a Python function to generate embeddings for documents and queries using the Gemini API. Set the `task_type` to `retrieval_document` for document embeddings and `retrieval_query` for query embeddings. These embeddings will be used to power a retrieval system. Refer to the API reference for more details on supported task types. 


In [9]:
from chromadb import Documents, EmbeddingFunction, Embeddings
from google.api_core import retry


class GeminiEmbeddingFunction(EmbeddingFunction):
    # Specify whether to generate embeddings for documents, or queries
    document_mode = True

    def __call__(self, input: Documents) -> Embeddings:
        if self.document_mode:
            embedding_task = "retrieval_document"
        else:
            embedding_task = "retrieval_query"

        retry_policy = {"retry": retry.Retry(predicate=retry.if_transient_error)}

        response = genai.embed_content(
            model="models/text-embedding-004",
            content=input,
            task_type=embedding_task,
            request_options=retry_policy,
        )
        return response["embedding"]

In [10]:
import warnings
warnings.filterwarnings('ignore')

In [11]:
import chromadb

DB_NAME = "googlerestaurentdb"
embed_fn = GeminiEmbeddingFunction()
embed_fn.document_mode = True

chroma_client = chromadb.Client()
db = chroma_client.get_or_create_collection(name=DB_NAME, embedding_function=embed_fn)

db.add(documents=documents, ids=[str(i) for i in range(len(documents))])

Here are the main points regarding the scope of our restaurant bot:

* **Customer-Facing Queries:**
    * Basic restaurant information.
    * Menu and ordering.
    * Reservations.
* **Customer Satisfaction:** Understanding terms related to food, service, ambiance, value, overall experience, dietary needs, order accuracy, and delivery.
* **Promotions/Deals & Events/Catering**
* **Feedback/Reviews & Loyalty Programs**
* **Allergens/Dietary Information & Sustainability**
* **Restaurant Staff & Equipment:** (Less frequent customer queries)

In [12]:
# Switch to query mode when generating embeddings.
embed_fn.document_mode = False

# Search the Chroma DB using the specified query.
query = "CAn you tell me menus"

result = db.query(query_texts=[query], n_results=1)
[[passage]] = result["documents"]

Markdown(passage)

Welcome to The Golden Spoon in Varanasi! We offer a diverse menu featuring the best of Indian, Chinese, Continental, South Indian, and Italian cuisines. Our opening hours are from 11:00 AM to 11:00 PM daily. For reservations or inquiries, please call us at 0542-XXXXXXX or visit our website at www.thegoldenspoon.com.

**Our Menu:**

**Indian Delights:**
Our Indian menu boasts popular dishes like creamy Butter Chicken (₹380), rich Paneer Tikka Masala (₹350), fragrant Chicken Biryani (₹420), flavorful Dal Makhani (₹300), and spicy Chana Masala (₹280). We also offer a variety of Indian breads including Naan (₹50), Roti (₹30), and Paratha (₹70). For seafood lovers, our Goan Fish Curry (₹450) is a must-try.

**Chinese Favorites:**
Indulge in our classic Chinese dishes such as Chicken Manchurian (dry/gravy - ₹320), Vegetable Fried Rice (₹250), Hakka Noodles (₹280), Schezwan Chicken (₹350), and Spring Rolls (₹180 for 2). We also offer a spicy Kung Pao Chicken (₹360) and a comforting Sweet and Sour Vegetables (₹290).

**Continental Selection:**
Our Continental menu features Grilled Chicken with Mushroom Sauce (₹400), Fish and Chips (₹380), Shepherd's Pie (₹360), Vegetable Au Gratin (₹320), and Chicken Stroganoff (₹420). These dishes offer a taste of global flavors prepared with our special touch.

**South Indian Specialties:**
Experience the authentic flavors of South India with our crispy Dosa (plain - ₹120, masala - ₹180, paneer - ₹220), fluffy Idli with Sambar (₹150 for 2), savory Vada (₹100 for 2), and tangy Pongal (₹160). Our South Indian Thali (₹300) offers a delightful assortment of regional specialties.

**Italian Indulgence:**
Savor our Italian classics including Margherita Pizza (₹350 for 10 inch), creamy Alfredo Pasta (with chicken - ₹400, with vegetables - ₹350), spicy Arrabbiata Pasta (₹320), cheesy Lasagna (₹450), and flavorful Mushroom Risotto (₹380). Our Garlic Bread (₹120 for 4 slices) is a perfect accompaniment.

**Beverages:**
We offer a range of beverages including Soft Drinks (₹60), Juices (₹80), Lassi (Sweet/Salted - ₹90, Mango - ₹120), Masala Chai (₹50), and Coffee (₹70).

**Desserts:**
Our dessert selection includes traditional Indian sweets like Gulab Jamun (₹80 for 2) and Rasgulla (₹70 for 2), as well as international favorites like Chocolate Lava Cake (₹200) and Ice Cream (₹100 per scoop).

**Vegetarian & Vegan Options:** We have a wide array of delicious vegetarian and vegan options across all cuisines, clearly marked on our in-house and online menus. Please ask your server for assistance.

**Allergies:** Please inform your server of any dietary restrictions or allergies, and our staff will be happy to assist you.

Enjoy your dining experience at The Golden Spoon! We appreciate your patronage.
